# Blue Rockfish Habitat Predictions

This notebook provides and training and prediction pipeline for habitat prediction. Please run the codes in order to properly train and make predictions. Outputs from this notebook will be saved to the scratch folder. Please add the final_predictions.shp file to arcgis pro to view the prediction. Prediction results are saved in the field "habitat_prediction"

In [2]:
! pip install geopandas 
! pip install xgboost 
! pip install scikit-learn 
! pip install simpledbf

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import os
import sys
import numpy as np
import matplotlib
import pandas as pd
import arcpy
import geopandas as gpd
import json
import pickle
import xgboost
import pickle
import xgboost as xgb
from xgboost import XGBClassifier
import sklearn
from sklearn.model_selection import GridSearchCV
from simpledbf import Dbf5

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


### Functions to train model

In [4]:
def preprocess_train_data(files):
    """
    Preprocess training data. Extract environmental information from the raster layers at observed and random points.
    Return sampled data file paths and their corresponding training labels (0 or 1).
    
    param files: a list of file paths. First path should be observed points, 
                 and the second path should be random points. The remanining paths should be 3-15 layers of rasters.
    """
    
    # set up env
    arcpy.env.workspace = '../../data'
    arcpy.env.overwriteOutput = True
    save_path = '../../scratch/'
    
    # gather the shape files
    files = np.array(files)
    obs = files[0]
    rnd = files[1]
    
    # gather the raster files
    rasters = files[2:]
    rasters = list(rasters[rasters != '#'])

    # sample rasters from observed points
    obs_table = save_path + 'observed_presence_sampled.dbf'
    obs_points = arcpy.sa.Sample(rasters, obs, obs_table)
    obs_count = int(arcpy.management.GetCount(obs_table)[0])
    obs_label = np.array([1]*obs_count)
    
    # sample rasters from absence points
    rnd_table = save_path + 'random_absence_sampled.dbf'
    rnd_points = arcpy.sa.Sample(rasters, rnd, rnd_table)
    rnd_count = int(arcpy.management.GetCount(rnd_table)[0])
    rnd_label = np.array([0]*rnd_count)

    return obs_table, obs_label, rnd_table, rnd_label

In [5]:
def build_model(files):
    """
    Build xgboost model with sampled points. Perform k-fold corss validation to find the best performing model.
    Output trained model in a pickle file and save to the scratch folders
    
    param files: a list of file paths. First path should be observed points, 
                 and the second path should be random points. The remanining paths should be 3-15 layers of rasters.
    """
    
    # preprocess data
    obs_path, obs_label, rnd_path, rnd_label = preprocess_train_data(files)
    
    # read training file
    obs_dbf = Dbf5(obs_path)
    rnd_dbf = Dbf5(rnd_path)
    obs_df = obs_dbf.to_dataframe()
    rnd_df = rnd_dbf.to_dataframe()

    # prepare data for training
    obs_df = obs_df.drop(columns = ['brf_obs', 'X', 'Y'])
    rnd_df = rnd_df.drop(columns = ['rand_obs', 'X', 'Y'])
    obs_df['label'] = obs_label
    rnd_df['label'] = rnd_label
    data = pd.concat([obs_df, rnd_df])
    X = data.drop(columns = ['label'])
    Y = data['label']
    
    # build xgboost model
    clf = XGBClassifier(objective= 'binary:logistic')
    parameters = {
        'max_depth': range (2, 10),
        'n_estimators': range(60, 220, 40),
        'learning_rate': [0.1, 0.01, 0.05]
    }
    grid_search = GridSearchCV(
        estimator = clf,
        param_grid = parameters,
        scoring = 'roc_auc',
        n_jobs = 5,
        cv = 5,
        verbose = True)
    grid_search.fit(X, Y)
    
    # print accuracy of model
    acc = sklearn.metrics.accuracy_score(grid_search.predict(X), Y)
    print('Training Accuracy is: ' + str(acc))
    
    # save model to scratch folder
    model_path = "../../scratch/xgb_model.pkl"
    pickle.dump(grid_search, open(model_path, "wb"))

### Functions to generate predictions

In [6]:
def preprocess_eval_data(files):
    """
    Preprocess prediction data. Extract environmental information from the raster layers at randomly generated points. 
    The number of generated points could be modified below. Return shape file with habitat prediction to scratch folder.
    
    param files: a list of file paths. The list should contain 3-15 layers of rasters.
    """
    # define env
    arcpy.env.workspace = 'scratch'
    arcpy.env.overwriteOutput = True
    
    # gather the raster files
    files = np.array(files)
    rasters = list(files[files != '#'])
    
    # create random points
    points = arcpy.management.CreateRandomPoints('../../scratch', 
                                        'prediction_points',  
                                        constraining_extent = rasters[0], 
                                        number_of_points_or_field = 300)
    
    
    # sample env information using the generated points
    out_table = '../../scratch/prediction_sampled.dbf'
    out_points = arcpy.sa.Sample(rasters, points, out_table)
    out_count = int(arcpy.management.GetCount(out_table)[0])
    out_label = np.array([1]*out_count)
    
    return out_table, out_label

In [7]:
def make_predictions(files):
    """
    This methods would generate predictions based on the file inputs. 
    Output from this method would be automatically saved to the scratch folder
    
    """
    arcpy.env.workspace = 'sratch'
    arcpy.env.overwriteOutput = True
    
    # preprocess data
    data, label = preprocess_eval_data(files)
    dbf = Dbf5(data)
    df = dbf.to_dataframe()
    X = df.drop(columns = ['prediction', 'X', 'Y'])
    
    # load pre-trained model
    model = pickle.load(open('../../scratch/xgb_model.pkl', 'rb'))
    predictions = model.predict(X)
    
    # Set local variables
    df['habitat_prediction'] = predictions
    
    # convert pd to gdf
    final_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.X, df.Y))
    
    # save gpd as shp
    final_gdf.to_file('../../scratch/final_predictions.shp')


### Run Training Pipeline

In [8]:
training_input_files = ['V:\\ENV859_Final_Project_al512\\data\\brf_obs.shp', 
                        'V:\\ENV859_Final_Project_al512\\data\\rand_obs.shp', 
                        'V:\\ENV859_Final_Project_al512\\data\\bathy', 
                        'V:\\ENV859_Final_Project_al512\\data\\habras10_1', 
                        'V:\\ENV859_Final_Project_al512\\data\\habras10_2', 
                        'V:\\ENV859_Final_Project_al512\\data\\habras10_3', 
                        'V:\\ENV859_Final_Project_al512\\data\\habras10_4', 
                        'V:\\ENV859_Final_Project_al512\\data\\habras10_5', 
                        'V:\\ENV859_Final_Project_al512\\data\\habras10_6', 
                        'V:\\ENV859_Final_Project_al512\\data\\habras10_7', 
                        'V:\\ENV859_Final_Project_al512\\data\\habras10_8', 
                        'V:\\ENV859_Final_Project_al512\\data\\dist_kelp', 
                        'V:\\ENV859_Final_Project_al512\\data\\dist_100m', 
                        'V:\\ENV859_Final_Project_al512\\data\\botc10_8ws', '#', '#']
build_model(training_input_files)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
Training Accuracy is: 0.9844559585492227


### Run Prediction Pipeline

In [9]:
pred_input_files = ['V:\\ENV859_Final_Project_al512\\data\\bathy', 
                    'V:\\ENV859_Final_Project_al512\\data\\habras10_1', 
                    'V:\\ENV859_Final_Project_al512\\data\\habras10_2', 
                    'V:\\ENV859_Final_Project_al512\\data\\habras10_3',
                    'V:\\ENV859_Final_Project_al512\\data\\habras10_4', 
                    'V:\\ENV859_Final_Project_al512\\data\\habras10_5', 
                    'V:\\ENV859_Final_Project_al512\\data\\habras10_6', 
                    'V:\\ENV859_Final_Project_al512\\data\\habras10_7', 
                    'V:\\ENV859_Final_Project_al512\\data\\habras10_8', 
                    'V:\\ENV859_Final_Project_al512\\data\\dist_kelp', 
                    'V:\\ENV859_Final_Project_al512\\data\\dist_100m', 
                    'V:\\ENV859_Final_Project_al512\\data\\botc10_8ws', '#', '#']

make_predictions(pred_input_files)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
